In [ ]:
import pandas as pd


UserAchievements = pd.read_csv('/kaggle/input/meta-kaggle/UserAchievements.csv')
TeamMemberships = pd.read_csv('/kaggle/input/meta-kaggle/TeamMemberships.csv')
Teams = pd.read_csv('/kaggle/input/meta-kaggle/Teams.csv')
Users = pd.read_csv('/kaggle/input/meta-kaggle/Users.csv', usecols=['Id', 'DisplayName'])

competition_gm_df = UserAchievements.query('AchievementType=="Competitions" and Tier==4')
gold_teams_df = Teams.query('Medal==1')[['Id', 'MedalAwardDate']]
TeamMemberships.columns = ['UId', 'Id', 'UserId', 'RequestDate']
gold_medal_history_df = pd.merge(TeamMemberships, gold_teams_df, on='Id', how='left').dropna()
gold_medal_history_df['MedalAwardDate'] = pd.to_datetime(gold_medal_history_df['MedalAwardDate'])
gold_medal_history_df = gold_medal_history_df.sort_values('MedalAwardDate')
competition_gm_df['TierAchievementDate'] = pd.to_datetime(competition_gm_df['TierAchievementDate'])
first_gold_df = gold_medal_history_df.groupby('UserId')['MedalAwardDate'].first().reset_index()
main = pd.merge(competition_gm_df, first_gold_df, on='UserId', how='left')
Users.columns = ['UserId', 'DisplayName']
main = pd.merge(main, Users, on='UserId', how='left')
main['day_from_first_gold_to_be_gm'] = main['TierAchievementDate'] - main['MedalAwardDate']

In [ ]:
pd.set_option('display.max_rows', 300)
main.sort_values('day_from_first_gold_to_be_gm')[['DisplayName', 'MedalAwardDate', 'TierAchievementDate', 'day_from_first_gold_to_be_gm']]